# Get repsonse from an LLM


To get started, [get an API key](https://g.co/ai/idxGetGeminiKey) and replace the word `API KEY` below with your API key:

In [72]:
import google.generativeai as genai
import nbimporter
from day0.main import getCurrentWeather, getLocation
from dotenv import load_dotenv
import os
import re


# Load environment variables from .env file
load_dotenv()

# Access the API key from the environment variable
api_key = os.getenv("SECRET_KEY")

genai.configure(api_key=api_key)
model = genai.GenerativeModel("gemini-1.5-flash")
# Combine the system prompt and user query into a single input because gemini does not take messages arrays like GPT!
full_prompt = f"{prompt}\n\nUser: {query}\nAssistant: {observation}\n\nUser:"
response = model.generate_content(full_prompt)
print(response.parts[0].text)


Thought: I have the user's location.  My next action should be to retrieve the current weather for that location.

Action: getCurrentWeather: New York City
PAUSE



Build your own Agent 

In [73]:
prompt = """
You cycle through Thought, Action, PAUSE, Observation. At the end of the loop you output a final Answer. Your final answer should be highly specific to the observations you have from running
the actions.
1. Thought: Describe your thoughts about the question you have been asked.
2. Action: run one of the actions available to you - then return PAUSE.
3. PAUSE
4. Observation: will be the result of running those actions.

Available actions:
- getCurrentWeather: 
    E.g. getCurrentWeather: Salt Lake City
    Returns the current weather of the location specified.
- getLocation:
    E.g. getLocation: null
    Returns user's location details. No arguments needed.

Example session:
Question: Please give me some ideas for activities to do this afternoon.
Thought: I should look up the user's location so I can give location-specific activity ideas.
Action: getLocation: null
PAUSE

You will be called again with something like this:
Observation: "New York City, NY"

Then you loop again:
Thought: To get even more specific activity ideas, I should get the current weather at the user's location.
Action: getCurrentWeather: New York City
PAUSE

You'll then be called again with something like this:
Observation: { location: "New York City, NY", forecast: ["sunny"] }

You then output:
Answer: <Suggested activities based on sunny weather that are highly specific to New York City and surrounding areas.>

"""

# Define your query
query = "What is the current weather at my location ?"

max_calls = 3
action_regex = r"^Action: (\w+): (.*)$"

# call the llm atleast 3 times 
for i in range(max_calls):
    print(f"Call {i + 1}:")

    # Split response text into lines
    response_lines = response.text.strip().split("\n")

    # Find the action string in the response
    found_action_str = next((line for line in response_lines if re.match(action_regex, line)), None)

    # Dummy functions to simulate API calls
    def getLocation(place=None):
        location = {
            "city": "New York City",
            "state": "NY",
            "country": "USA"
        }
        return location

    def getCurrentWeather(place=None):
        weather = {
            "temperature": "2",
            "unit": "F",
            "forecast": "snowy"
        }
        return weather

    # Map available functions
    available_functions = {
        "getLocation": getLocation,
        "getCurrentWeather": getCurrentWeather
    }

    # Execute the identified action if it matches
    actions = re.match(action_regex, found_action_str) if found_action_str else None
    if actions:
        action, action_arg = actions.groups()
        if action in available_functions:
            observation = available_functions[action](action_arg)
            print("Observation:", observation)
        else:
            print(f"Action '{action}' is not available.")
    else:
        print("No valid action found.")



Call 1:
Observation: {'temperature': '2', 'unit': 'F', 'forecast': 'snowy'}
Call 2:
Observation: {'temperature': '2', 'unit': 'F', 'forecast': 'snowy'}
Call 3:
Observation: {'temperature': '2', 'unit': 'F', 'forecast': 'snowy'}
